# Visualizing Genres using Clustering Models

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

## Data Pre-processing

In [ ]:
spotify = pd.read_csv('../data/tops/features.csv',index_col=0)
spotify.dropna(how='any', inplace=True)
spotify.head()

In [ ]:
features =['danceability','energy','key','loudness', 'mode', 
           'speechiness', 'acousticness', 'instrumentalness', 
           'liveness','valence','tempo','duration_ms', 'time_signature']

x = spotify.loc[:,features].values
x = StandardScaler().fit_transform(x)

## Principal Component Analysis

In [ ]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(x)

principal_df = pd.DataFrame(data=principal_components, columns=['principal_component_1', 'principal_component_2'])

kmeans_pca = KMeans(n_clusters=8, random_state=0, init='random').fit(principal_df)
predicted_pca = kmeans_pca.fit_predict(music_data_pca)

principal_df['prediction'] = predicted_pca
principal_df.set_index(keys=spotify.index, inplace=True)
principal_df.head()

In [ ]:
spotify['pca_pred'] = predicted_pca

## t-SNE

In [ ]:
tsne = TSNE(n_components=2)
x_embedded = tsne.fit_transform(x)

In [ ]:
x_embedded

In [ ]:
tsne = TSNE(n_components=2)
x_embedded = tsne.fit_transform(x)
tsne_df = pd.DataFrame(data=x_embedded, columns=['tsne_1', 'tsne_2'])

kmeans_tsne = KMeans(n_clusters=8, random_state=0, init='random').fit(tsne_df)
predicted_tsne = kmeans_tsne.fit_predict(tsne_df)

tsne_df['prediction'] = predicted_tsne
tsne_df.set_index(keys=spotify.index, inplace=True)
tsne_df.head()

In [ ]:
spotify['tsne_pred'] = predicted_tsne

## Grouped DataFrames

In [ ]:
pred_0_pca = principal_df[principal_df['prediction'] == 0]
pred_1_pca = principal_df[principal_df['prediction'] == 1]
pred_2_pca = principal_df[principal_df['prediction'] == 2]
pred_3_pca = principal_df[principal_df['prediction'] == 3]
pred_4_pca = principal_df[principal_df['prediction'] == 4]

pred_0_tsne = tsne_df[tsne_df['prediction'] == 0]
pred_1_tsne = tsne_df[tsne_df['prediction'] == 1]
pred_2_tsne = tsne_df[tsne_df['prediction'] == 2]
pred_3_tsne = tsne_df[tsne_df['prediction'] == 3]
pred_4_tsne = tsne_df[tsne_df['prediction'] == 4]

## Comparison Plot

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
sns.scatterplot(x='principal_component_1', y='principal_component_2', data=principal_df, 
                hue='prediction', palette=sns.color_palette('hls',8), 
                style='prediction', markers=["X","X","X","X","X","X","X","X"])

for center, i in zip(kmeans_pca.cluster_centers_, range(8)):
    if not i:
        plt.scatter(center[0], center[1], marker='*',s=200, c='black', label='Centroid')
    else:
        plt.scatter(center[0], center[1], marker='*',s=200, c='black')
    
plt.xlabel('First Principal Component (PCA)')
plt.ylabel('Second Principal Component (PCA)')
plt.title('k-Means Clustering via PCA', pad=20)
plt.legend(shadow=True)

plt.subplot(1,2,2)
sns.scatterplot(x='tsne_1', y='tsne_2', data=tsne_df, 
                hue='prediction', palette=sns.color_palette('hls',8),
                style='prediction', markers=['X','X','X','X','X','X','X','X'])

for center, i in zip(kmeans_tsne.cluster_centers_, range(8)):
    if not i:
        plt.scatter(center[0], center[1], marker='*',s=200, c='black', label='Centroid')
    else:
        plt.scatter(center[0], center[1], marker='*',s=200, c='black')

plt.xlabel('t-SNE One')
plt.ylabel('t-SNE Two')
plt.title('k-Means Clustering via t-SNE Reduction', pad=20)
plt.legend(shadow=True)
plt.show();